**CÀI ĐẶT VÀ IMPORT THƯ VIỆN**

In [ ]:
!pip install catalyst==20.2.4
!pip install tqdm==4.33
!pip install efficientnet_pytorch
!pip install pytorch_toolbelt

In [ ]:
from sklearn import svm
from keras.callbacks import CSVLogger
from keras.callbacks import LambdaCallback
from utils.utils import *
import pandas as pd
import sys
import os
import cv2
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import numpy as np
from numpy import asarray
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.callbacks import LambdaCallback
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from torch.utils.data import Dataset, DataLoader
import tensorflow as tf
import torch
import collections
from pytorch_toolbelt.inference import tta
from catalyst.dl.callbacks import InferCallback
from catalyst.dl.runner import SupervisedRunner
from torch.nn.functional import softmax
from catalyst.dl.callbacks import EarlyStoppingCallback, AccuracyCallback, F1ScoreCallback, ConfusionMatrixCallback, MixupCallback
from catalyst.contrib.nn.schedulers import OneCycleLR, ReduceLROnPlateau, StepLR, MultiStepLR
from torchvision import transforms
import torch.nn.functional as F
from torch.autograd import Variable
from torchsummary import summary
import pickle
import time
import copy

In [ ]:
splits = pickle.load(open('./utils/cv_split.pickle', 'rb'))
data = pd.read_csv('./utils/splited_train_new.csv')
labels  = ['N','D','G','C','A','H','M','O']
n_classes = len(labels)
fold_idx, batch_size, model_name, image_size, head_n_epochs, head_lr, full_n_epochs, full_lr = 3, 32, 'efficientnet-b3', 240, 5, 1e-5, 15, 3e-4
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
num_classes = len(labels)
seed_everything(1234)
runner = SupervisedRunner()
model = prepare_model(model_name, n_classes)

**TRAIN ẢNH CLAHE**

In [ ]:
train_path = './data/clahe-train/'
valid_path = './data/clahe-train/'
train_dataset = EyeDataset(dataset_path = train_path, 
					labels=data.loc[splits['train_idx'][fold_idx],labels].values, 
					ids=data.loc[splits['train_idx'][fold_idx],'id'].values, 
					albumentations_tr=aug_train_heavy(image_size)) 
val_dataset = EyeDataset(dataset_path=valid_path, 
					labels=data.loc[splits['test_idx'][fold_idx],labels].values, 
					ids=data.loc[splits['test_idx'][fold_idx],'id'].values, 
					albumentations_tr=aug_val(image_size))
train_loader =  DataLoader(train_dataset,
					num_workers=8,
					pin_memory=False,
					batch_size=batch_size,
					shuffle=True)
val_loader =  DataLoader(val_dataset,
					num_workers=8,
					pin_memory=False,
					batch_size=batch_size,
					shuffle=True)	
loaders = collections.OrderedDict()
loaders["train"] = train_loader
loaders["valid"] = val_loader
logdir = 'logs/'

In [ ]:
print('Training only head for {} epochs with inital lr {}'.format(head_n_epochs, head_lr))
for p in model.parameters():
		p.requires_grad = False
for p in model._fc.parameters():
		p.requires_grad = True
optimizer = torch.optim.Adam(model.parameters(), lr=head_lr)
criterion = nn.BCEWithLogitsLoss()
scheduler = ReduceLROnPlateau(optimizer=optimizer, factor=0.75, patience=2)
runner.train(model=model,
			criterion=criterion,
			optimizer=optimizer,
			loaders=loaders,
			logdir=logdir,
			scheduler=scheduler,
			callbacks=[
                MixupCallback()
				],
			num_epochs=head_n_epochs,
			verbose=True)      
print('Train whole net for {} epochs with initial lr {}'.format(full_n_epochs, full_lr))
for p in model.parameters():
		p.requires_grad = True
optimizer = torch.optim.Adam(model.parameters(), lr=full_lr)
criterion = nn.BCEWithLogitsLoss()
scheduler = ReduceLROnPlateau(optimizer=optimizer, factor=0.5, patience=5)
runner.train(model=model,
			criterion=criterion,
			optimizer=optimizer,
			loaders=loaders,
			logdir=logdir,
			scheduler=scheduler,
			callbacks=[
                MixupCallback()
				],
			num_epochs=full_n_epochs,
			verbose=True)

**TEST CLAHE**

In [ ]:
test_data = pd.read_csv('./utils/XYZ_ODIR.csv')
test_data_left = test_data.copy()
test_data_right = test_data.copy()
test_data_left.loc[:,'id'] = test_data_left.ID.apply(lambda x: str(x)+'_left.jpg')
test_data_right.loc[:,'id'] = test_data_left.ID.apply(lambda x: str(x)+'_right.jpg')
test_data = pd.concat([test_data_left,test_data_right])
test_data.sort_values(['ID'],inplace=True)
test_path = './data/clahe-test/'
test_dataset = EyeDataset(dataset_path=test_path, 
                         labels=test_data.loc[:,labels].values, 
                         ids=test_data.loc[:,'id'].values, 
                         albumentations_tr=aug_val(image_size))
test_loader =  DataLoader(test_dataset,
                         num_workers=8,
                         pin_memory=False,
                         batch_size=batch_size,
                         shuffle=False)  
loaders = collections.OrderedDict()
loaders["valid"] = test_loader    
probabilities_list = []
ttatype='d4'
logdir = 'logs/'
model = prepare_model(model_name, n_classes)
model.cuda()
model.load_state_dict(torch.load(os.path.join(logdir, './logs/clahe.pth'))['model_state_dict'])
model.eval()
if ttatype == 'd4':
    model = tta.TTAWrapper(model, tta.d4_image2label)
elif ttatype == 'fliplr_image2label':
    model = tta.TTAWrapper(model, tta.d4_image2label)
runner = SupervisedRunner(model=model)
runner.infer(model=model,loaders=loaders,callbacks=[InferCallback()])
predictions = runner.callbacks[0].predictions['logits']
probabilities = softmax(torch.from_numpy(predictions),dim=1).numpy()    	
for idx in range(probabilities.shape[0]):	
    if all(probabilities[idx,:]<0.5):	
        probabilities[idx,0] = 1.0	
probabilities_list.append(probabilities)	
probabilities_combined = np.stack(probabilities_list,axis=0).mean(axis=0) 	
predicted_labels = pd.DataFrame(probabilities_combined, columns=labels)	
predicted_labels['id'] = test_data.loc[:,'id'].values	
predicted_labels.loc[:,'ID'] = predicted_labels.id.apply(lambda x: x.split('_')[0])	
predicted_labels_groupped = predicted_labels.groupby(['ID']).aggregate(dict(zip(labels,['max']*(len(labels)))))	
print(type(predicted_labels_groupped))
predicted_labels_groupped['ID'] = predicted_labels_groupped.index.values.astype(int)	
predicted_labels_groupped.reset_index(drop=True, inplace=True)	
predicted_labels_groupped.sort_values('ID',inplace=True)	
predicted_labels_groupped = predicted_labels_groupped.loc[:,['ID']+labels] 	
predicted_labels_groupped.to_csv('./submit/clahe.csv', index=False)